
# reddit-wiki-ripper
by Jam#0001

First things first, install the required packages:

In [1]:
!pip3 install praw regex


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the 'c:\python38\python.exe -m pip install --upgrade pip' command.


Go to the [reddit apps section](https://www.reddit.com/prefs/apps/) and create a new app.

![app creation image](https://i.imgur.com/REpuvlH.png)

Set the following as the client ID and secret

![client ID and secret indicated image](https://i.imgur.com/kqdoRvW.png)

In [2]:
client_id = input("Client ID: ")

In [3]:
client_secret = input("Client Secret: ")

Now input the subreddit you wish to grab the wiki from

In [4]:
subreddit = input("Please enter a subreddit: ")

Last step! Time to grab those wiki pages: 

In [5]:
import os
import regex
import praw

reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    user_agent="reddit_wiki_ripper"
)

# if not os.path.exists(os.path.join("./", subreddit)):
os.mkdir(os.path.join("./", subreddit))

for wikipage in reddit.subreddit(subreddit).wiki:
    # if the subreddit is nested
    if wikipage.name.find("/") != -1:
        dir = os.path.join(os.path.join("./", subreddit), wikipage.name[:wikipage.name.rindex('/')])
        if not os.path.exists(dir):
            os.makedirs(dir)
    file_dir = os.path.join(os.path.join("./", subreddit), wikipage.name)
    with open("{}.md".format(file_dir), "w", encoding="utf-8") as f:
        # fix headings
        formatted_wikipage = regex.sub(r'(?<=\n#?)#(?=[^# ])', "# ", wikipage.content_md)
        # replace links so they are to localhost rather than to reddit
        formatted_wikipage = formatted_wikipage.replace("https://www.reddit.com/r/{}/wiki/".format(subreddit), "./")
        f.write(formatted_wikipage)

If you are using google collab, run the following script to download all the files as a zip

In [6]:
import zipfile

zipf = zipfile.ZipFile('wiki.zip', 'w', zipfile.ZIP_DEFLATED)
for root, dirs, files in os.walk(os.path.join('./', subreddit)):
    for file in files:
        zipf.write(os.path.join(root, file))
zipf.close()

from google.colab import files
files.download("/content/wiki.zip")